In [ ]:
!nvidia-smi

Mon Jul  4 09:25:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
drive.mount('/content/gdrive')
!ls
%cd gdrive/My\ Drive/MalMem/
!ls

Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/MalMem
 adasyn_tl_xgboost_balance.ipynb
 adasyn_tl_xgboost_balance_noRUS.ipynb
 adasyn_tl_xgboost_imbalanced.ipynb
 clf_readcsv.py
 CNN_MalMem_balance
 CNN_MalMem_imbalance
 createCNN_Balanced.ipynb
 createCNN_Balanced_noRUS.ipynb（副本）
 createCNN_Imbalanced.ipynb
'Dynamic Malware Analysis with Feature Engineering and Feature Learning.pdf'
 ensemble_major_imb.ipynb
 ensemble_major.ipynb
 evaluate_auc_3Algorithm_micro.ipynb
 Evaluate_PR_3Algorithms.ipynb
 Evaluate_PR_CNN.ipynb
 Evaluate_PR_Voting_imb.ipynb
 Evaluate_PR_Voting.ipynb
 evaluate_PR_xgboost_imb.ipynb
 evaluate_PR_xgboost.ipynb
 evaluation_MM.xlsx
 logs
 MalMem_Test_x.csv
 MalMem_Test_y.csv
 MalMem_Train_x.csv
 MalMem_Train_y.csv
 MalMem_x.csv
 MalMem_y.csv
'Malware Memory Analysis _ Datasets _ Canadian Institute for Cybersecurity _ UNB.mhtml'
 myXgb_adasyn_MalMem_imb.json
 myXgb_adasyn_MalMem.json
 Obfuscated-MalMem2022.csv
 Obfuscated-MalMem2022-excel.xlsx

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  8 11:13:03 2022

load adasyn and xgbboost balanced model, test on 20% test dataset and evaluate
try using ROC curve
@author: Administrator
"""
# import joblib
import pandas as pd
import xgboost as xgb
from sklearn.metrics import confusion_matrix, zero_one_loss, accuracy_score, \
    classification_report, roc_auc_score, roc_curve, auc, plot_confusion_matrix, ConfusionMatrixDisplay
from collections import Counter
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from imblearn.metrics import geometric_mean_score, classification_report_imbalanced
# train_x = pd.read_csv('KDDTrain_x_20.csv',header=None)
# train_Y = pd.read_csv('KDDTrain_y_20.csv',header=None)
# train_Y[0]=list(train_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))

# print("train_x", train_x.head())
# print("train_y", train_Y.head())


#testing dataset 20%
test_x = pd.read_csv('MalMem_Test_x.csv',header=None)
test_Y = pd.read_csv('MalMem_Test_y.csv',header=None)

#test_Y[0]=list(test_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))
print("test_x", test_x.head())
print("test_y", test_Y.head())

test_Y  = test_Y.values.ravel()
print('original dataset shape %s' % Counter(test_Y))

# load model voting
import joblib
clf = joblib.load("VotingClassifier_MalMem_imbalance.pkl")

# after tomek link 
#pred_y = clf.predict(test_x)  #x_res
"""
results = confusion_matrix(test_Y, pred_y)   #Y_res
error = zero_one_loss(test_Y, pred_y)   #Y_res
target_names = ['benign 0', 'dos 1', 'probe 2', 'r2l 3', 'u2r 4']
print("20% result: \n" , results)
print("20% error:", error)
print("20% validation:", accuracy_score(test_Y, pred_y))   #Y_res
"""
yy = clf.predict_proba(test_x)  #x_res
# [:, 1]
# yy = clf.decision_function(x_res)[:, 1]
"""
rac = roc_auc_score(test_Y, yy, multi_class='ovr', average ='macro')  #Y_res
print("ROC AUC Score:", rac)
print(classification_report(test_Y, pred_y, target_names=target_names))   #Y_res
print(classification_report_imbalanced(test_Y, pred_y, target_names=target_names))  #Y_res
print(f"The geometric mean is {geometric_mean_score(test_Y, pred_y):.3f}") #Y_res
#import seaborn as sns

#sns.set_context("poster")

# disp = plot_confusion_matrix(clf, test_x, test_Y, colorbar=False)  #x_res, Y_res

cm = confusion_matrix(test_Y, pred_y, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=clf.classes_)
disp.plot()
_ = disp.ax_.set_title("XGBoost")
plt.show()
# ***plot confusion matrix
# plt.matshow(results, cmap=plt.cm.gray)
# plt.show()

"""
from sklearn.preprocessing import label_binarize
# plot PR curve
y_bin = label_binarize(test_Y, classes=[0, 1, 2, 3])  #Y_res
n_classes = y_bin.shape[1]

print("y_bin:", y_bin)
print("n_classes:", n_classes)

#lw = 4


test_x          0         1         2         3         4         5         6   \
0 -0.587615  0.107713 -0.778478 -0.302401 -0.972787 -0.783229 -0.439421   
1 -0.068333 -1.021497  1.290864  0.526254  1.025516  1.517175  0.473084   
2  0.104762  0.484116 -0.559920 -0.285955 -0.508841 -0.799160 -0.247715   
3 -0.068333 -0.268691  0.858378  0.366127  0.725316  1.095824  0.322267   
4 -0.241427  0.484116 -1.175315 -0.388084 -0.933367 -1.068066 -0.430380   

         7         8         9   ...        40        41        42        43  \
0 -0.245733 -0.076951 -0.885724  ... -0.484390  0.194377 -0.518261 -0.204202   
1  0.389141  0.055321  1.036174  ... -0.672147  0.194377  0.806340  0.298040   
2 -0.196961 -0.013437 -0.672180  ... -0.341833  0.194377 -0.518261 -0.204202   
3  0.266459  0.043667  0.691027  ... -0.095600  0.194377  0.806340  0.298040   
4 -0.311379 -0.071998 -0.875792  ...  0.643100  0.194377 -0.518261 -0.204202   

         44        45        46        47        48        49

In [4]:
# %%
# The average precision score in multi-label settings
# ...................................................
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_bin[:, i], yy[:, i])
    average_precision[i] = average_precision_score(y_bin[:, i], yy[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(
    y_bin.ravel(), yy.ravel()
)
average_precision["micro"] = average_precision_score(y_bin, yy, average="micro")

In [6]:
# %%
# Plot Precision-Recall curve for each class and iso-f1 curves
# ............................................................
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import PrecisionRecallDisplay
# setup plot details
colors = cycle(["navy", "turquoise", "darkorange", "cornflowerblue"])  #, "teal"])

_, ax = plt.subplots(figsize=(7, 8))

f_scores = np.linspace(0.2, 0.8, num=4)
lines, labels = [], []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    (l,) = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
    plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))

display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
)
display.plot(ax=ax, name="Micro-average precision-recall", color="gold")

for i, color in zip(range(n_classes), colors):
    display = PrecisionRecallDisplay(
        recall=recall[i],
        precision=precision[i],
        average_precision=average_precision[i],
    )
    display.plot(ax=ax, name=f"Precision-recall for class {i}", color=color)

# add the legend for the iso-f1 curves
handles, labels = display.ax_.get_legend_handles_labels()
handles.extend([l])
labels.extend(["iso-f1 curves"])
# set the legend and the axes
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.legend(handles=handles, labels=labels, loc="best")
ax.set_title("Voting Classifier Precision-Recall curve of multi-class")

plt.show()